# N-Grams: conceptually very simple language models
N-Grams store for each CONTEXT (of size N-1), what other tokens have been observed in some training data and how often they have been observed.

For example, in a given training data, "The" could be followed by "y" once, "n" twice, and a blank (" ") 127 times (no other characters follow this context). From this, we can derive our probability density function: We'd say that P(y|the)=1/130, P(n|the) = 2/130, P(" "|the)=127/130 and all others are 0.

These numbers end up being quite accurate for large amounts of training data and small N (say, up to 5 or 7).

N-Grams are very compute-efficient and (if done right) reasonably memory-efficient. 

In the code below, we use characters as tokens; such a model can e.g. be useful for correcting spelling mistakes.

In [23]:
def ngrams_from_text(f, N):
    '''
    Given a text file (f), create all pairs of (context,next) in that file.
    This code is ignorant of line starts and endings and considers all the data as one string
    '''
    context = list(f.read(N - 1))
    last = f.read(1)
    while last != '':
        yield context, last
        context = context[1:] + list(last)
        last = f.read(1)

In [24]:
def get_base_node(model, prefix):
    """from our model tree, get the node that represents the prefix (above: context) that we want to append to"""
    node = model
    for c in prefix:
        if c not in node:
            node[c] = {}
        node = node[c]
    return node


def add_ngram_to_model(model, prefix, last):
    base = get_base_node(model, prefix)
    if last not in base:
        base[last] = 0
    base[last] += 1


def estimate_model(source, N):
    model = {}
    with open(source, 'r', encoding='utf-8') as f:
        ngram_source = ngrams_from_text(f, N)
        for prefix, last in ngram_source:
            add_ngram_to_model(model, prefix, last)
    return model

In [42]:
N = 9
source = "data/shakespeare-en.txt" # other files in data: lyrik-de.txt, dialoge-de.txt, merkel-de.txt
model = estimate_model(source, N)

In [ ]:
model # run this only if you want to see a lot of output, especially for large N

In [29]:
import random # we'll need this to sample from the distribution (random.choices)


def generate(model, start):
    start = list(start)
    while True:
        base = get_base_node(model, start)
        chars, counts = zip(*base.items())
        char_list = random.choices(population=chars, weights=counts, k=1)
        start = start[1:] + char_list
        yield char_list[0]

In [49]:
PROMPT = "What is the color of the sky"
print(PROMPT, end='')

# limit context to the correct size
if (len(PROMPT) > N-1):
    CONTEXT = PROMPT[len(PROMPT)-N+1:]
else:
    CONTEXT = " " * (N-1-len(PROMPT)) + PROMPT
i = 0

# now generate 1000 characters using weighted sampling
for c in generate(model, CONTEXT):
    print(c, end='')
    i += 1
    if i > 1000:
        break
print()

What is the color of the sky, the weaver. This
    they have access to you! Hold, hold, my lord,
    Which writ hath despis'd! traitorously as I may, indeed? Shall we burn daylight see my shame,
    Who kept him a paper-mill. It holds his horns.
  HOLOFERNES. I do not know me one side so, against any man in his life, and all.
  DUCHESS OF YORK

  QUEEN. What it should justice of it,  
    For thee, and her son, as thou com'st not speak England shall ne'er pays after-love I do commit into the King
    We shall say good night, I was no queen
  HELENA. I do protest unto the heart out ere the devil come to her friend, and the truth,
    That any villany!
  Claud. I have, you
    Immoment thrown upon his grandsire's tomb,
    To sell again bestride him his entrails were half so kind a father.  
  BOLINGBROKE, Duke of Norfolk, Thomas Gargrave, hast the sparrow. This
    was again;
    And in heaven, my blood,
    To him that fleec'd poor part,
    The Presented. Now 'tis here. On th' market-

Note that the N-gram fails when we query it with a context that it has never seen. (e.g., it hasn't seen "sky?" with question mark). 
In typical applications, we either back-off to a N-1 n-gram, or we have smoothed the model so that it is able to compute probability density functions for all possible contexts. 

Note also, that generation does not yet contain a "temperature" parameter (it always uses a temperature of 1.0). 